# Streaming Data From A File To Power BI Streaming Dataset Using Python
> Create streaming dataset based on an existing file using Python. Super handy for POCs.

- toc: true 
- badges: true
- comments: true
- categories: [powerbi, pandas, python, api, streaming, dataset]
- hide: false

## Streaming Dataset

It's easy to create a streaming dataset in Power BI for real-time applications. You do not need a premium capacity. To keep the blog short, I am going to assume you already know what is a streaming dataset and how it differs from import/DQ. If you are not familiar, [this documentation](https://learn.microsoft.com/en-us/power-bi/connect-data/service-real-time-streaming) will help. 

Imagine you are creating a POC and would like to develop the real-time dashboard, report or just want to show to your stakeholders what the solution would loook like, you will need data to stream to Power BI service. You can create Azure Event Hub, Streaming Analytics. But in this blog, I will show you how you can stream data from an existing file. I will also include a link below in the resources for a few sample datasets you can use depending on the use case. 

You will need a Python IDE with pandas installed.



##### Create A Streaming dataset

First create a streaming dataset in Power BI service using the API method.
![](https://raw.githubusercontent.com/pawarbi/blog/master/images/KP9NMM8b4g.png)

I will be using this dataset for my example. Note the columns and column types from your data and create the dataset. 
![](https://raw.githubusercontent.com/pawarbi/blog/master/images/msedge_dyzzbjo5kF.png)

Note the API endpoint:
![](https://raw.githubusercontent.com/pawarbi/blog/master/images/msedge_imlnDaqdcW.png)

### Python Script

## Import libraries 

import requests
from datetime import datetime
from dataclasses import dataclass
import numpy as np
from typing import List, Dict
from time import sleep
import pandas as pd

In [ ]:
import requests
from datetime import datetime
from dataclasses import dataclass
import numpy as np
from typing import List, Dict
from time import sleep
import pandas as pd

In [ ]:


@dataclass


class StreamToPowerBI:
    
    """

    Class to stream data to Power BI Streaming dataset
    --------------------------------------------------
    Example:

     - endpoint :  API ednpoint, e.g. "https://api.powerbi.com/beta/<ws_id>/datasets/<dataset_id>/rows?key=<___key___>"
     - data     :  Data in dictionary e.g. {"id":1, "Value":2.33, "Color":"Red"} 
                   Number of columns, column names and column type must match with Power BI.
     - delay    :  Delay in seconds in sending the data to Power BI. e.g. if delay is 3, data will be sent every 3 seconds.
                   default value is 2 seconds. To overwrite, use the wait() method
                   
                   
     example:      To stream data to the specified endpoint with 5 s delay
                   StreamToPowerBI(endpoint,data).wait(5).post_data()              
 
    """
    endpoint: str
    data: Dict
    delay: int=2   
        
    def wait(self, other=None):
        ''' Delay in seconds to stream the data '''
        if other==None:
            # default value is 2 seconds
            other = 2 
        sleep(other)    
        return self
    
    def post_data(self):
        ''' Use post method send the data to Power BI'''
        if isinstance(self.data, dict):
            endpoint  = self.endpoint
            payload = [self.data]

            x = requests.post(endpoint, json = payload)
            if x.status_code==200:
                x=x
            else:
                print("Failed ! Check endpoint, data")
        else:
            print("Data passed is not a dictionary. It should be in the format {}".format('{}'))
        return x
    
    
    

##### Data

Import the data using pandas or any other library (spark, polars etc.). Be sure to make the numeric columns type `float` otherwise json serialization will fail. I am also removing null values.

In [ ]:
df = (pd.read_csv("https://raw.githubusercontent.com/reisanar/datasets/master/eBayAuctions.csv", 
                  index_col=None, 
                  dtype={'Category':str,
                         'currency':str,
                         'sellerRating':float,
                         'Duration':float,
                         'ClosePrice':float,
                         'OpenPrice':float,
                         'Competitive?':float},
                  usecols=['Category','currency','sellerRating','Duration','ClosePrice','OpenPrice','Competitive?'])
     ).dropna()

##### Test 

Before streaming data from the whole file, let's first test using one data point. Note that in the orginal data, there is no timestamp, so I am adding the timestamp , i.e `DateTime` column, to the dictionary. In your case, you may not want to do that. comment it out if you want to stream the data as is. 

In [ ]:
d = dict(df.iloc[0])
d['DateTime']=datetime.now().isoformat()
d